In [1]:
import torch
import torch.nn as nn

## **继承Moudule类构造模型**
Module类是所有的神经网络模块的基类，可以继承Module类来构造模型
- \__init\__定义参数
- forward函数定义计算     
示例:构建多层感知机

In [2]:
class MLP(nn.Module):
    def __init__(self, **kwargs):
        super(MLP, self).__init__(**kwargs)
        self.hidden = nn.Linear(784, 256)
        self.act = nn.ReLU()
        self.output = nn.Linear(256, 10)
    
    # 定义前向计算的过程
    def forward(self, x):
        a = self.act(self.hidden(x))
        return self.output(a)

In [3]:
X = torch.randn((2, 784))

In [4]:
net = MLP()
net(X)

tensor([[ 3.8471e-03, -3.0670e-04,  2.6725e-01, -6.8623e-02,  3.8541e-01,
         -8.0052e-02,  8.7929e-02,  3.4679e-02,  1.5007e-01, -2.1541e-01],
        [ 9.8713e-02, -4.9671e-01, -2.0296e-01, -1.2435e-02,  3.9010e-01,
          3.0062e-02, -1.1622e-01,  7.7738e-02, -6.3190e-02, -2.4741e-01]],
       grad_fn=<AddmmBackward>)

## **利用Sequential类构建模型**
Sequential类继承自Module类，接受一个由各层组成的OrderedDict或者子模块序列作为模型输入
- 模型是简单的连接计算     
接下来实现一个简单的Sequential类

In [12]:
class MySequential(nn.Module):
    from collections import OrderedDict
    def __init__(self, *args):
        super(MySequential, self).__init__()
        if len(args) == 1 and isinstance(args[0], OrderedDict):
            # 传入的是一个OrderDict
            for key, module in args[0].items():
                # 将module添加入self._modules
                self.add_module(key, module)
        else:
            # 传入的是一些Module
            for idx, module in enumerate(args):
                self.add_module(str(idx), module)
            
    def forward(self, x):
        # self._modules是一个OrderDict
        for module in self._modules.values():
            x = module(x)
        return x

In [13]:
net = MySequential(nn.Linear(784, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.1277,  0.2611,  0.3723,  0.1648, -0.1378, -0.5152, -0.0593, -0.1710,
          0.3450,  0.0930],
        [ 0.0038, -0.0792,  0.1885,  0.0065, -0.1644, -0.2313, -0.1212,  0.2929,
          0.0378, -0.1411]], grad_fn=<AddmmBackward>)

## **利用ModuleList构建模型**
ModuleList接受一个Module组成的list来构建模型，具有extend和append等list具有的操作

In [15]:
net = nn.ModuleList([nn.Linear(784, 256), nn.ReLU(), nn.Linear(256, 10)])
net[-1]

Linear(in_features=256, out_features=10, bias=True)

类似的还有ModuleDict
> 以上两个类没有实现forward方法，所以不能用来作为模型，只能用于构建模型

## **复杂模型**
nn和nn.functional的区别：nn中的都是类，nn.functional里的都是函数

In [23]:
class FancyMLP(nn.Module):
    def __init__(self, **kwargs):
        super(FancyMLP, self).__init__(**kwargs)
        self.rand_weight = torch.rand((20, 20), requires_grad=False) # 这个参数不参与训练
        self.linear = nn.Linear(20, 20)
    
    def forward(self, x):
        x = self.linear(x)
        
        x = nn.functional.relu(torch.mm(x, self.rand_weight.data) + 1)
        
        x = self.linear(x) # 此处使用了同样的属性，等于是复用层，共享参数
        
        while x.norm().item() > 1:
            x /= 2
        if x.norm().item() < 0.8:
            x *= 10
        return x.sum()

以上模型含有几个自定义的地方
- 层的复用，参数共享
- 部分参数不训练

In [24]:
X = torch.rand(2, 20)
net = FancyMLP()
print(net)
net(X)

FancyMLP(
  (linear): Linear(in_features=20, out_features=20, bias=True)
)


tensor(-0.5368, grad_fn=<SumBackward0>)